<a href="https://colab.research.google.com/github/troberts60/LRRK2_Drug_Study/blob/main/LRRK2_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install ChEMBL

In [ ]:
! pip install chembl_webresource_client

     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 596 kB 20.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


##Import Libraries

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

##Search LRRK2

In [ ]:
target = new_client.target
target_query = target.search('LRRK2')
all_targets = pd.DataFrame.from_dict(target_query)
all_targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Mus musculus,Leucine-rich repeat serine/threonine-protein k...,17.0,False,CHEMBL2010622,"[{'accession': 'Q5S006', 'component_descriptio...",SINGLE PROTEIN,10090
1,"[{'xref_id': 'Q5S007', 'xref_name': None, 'xre...",Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,16.0,False,CHEMBL1075104,"[{'accession': 'Q5S007', 'component_descriptio...",SINGLE PROTEIN,9606


In [ ]:
sele_target = all_targets.target_chembl_id[1]
sele_target

'CHEMBL1075104'

In [ ]:
bio_act = new_client.activity
info = bio_act.filter(target_chembl_id=sele_target).filter(standard_type="IC50")

In [ ]:
df1 = pd.DataFrame.from_dict(info)
df1

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,6180232,[],CHEMBL1772597,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '16.63', 'le': '0.30', 'lle': '1.48', ...",CHEMBL1771409,None,CHEMBL1771409,4.88,False,http://www.openphacts.org/units/Nanomolar,1420091,=,1,True,=,None,IC50,nM,None,13200.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,13.2
1,None,6180234,[],CHEMBL1772599,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '20.54', 'le': '0.37', 'lle': '2.01', ...",CHEMBL1771411,None,CHEMBL1771411,5.39,False,http://www.openphacts.org/units/Nanomolar,1420093,=,1,True,=,None,IC50,nM,None,4100.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,4.1
2,None,7989381,[],CHEMBL1937075,Inhibition of human LRRK2,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,None,None,CHEMBL1933005,Bioorg. Med. Chem. Lett.,2012,"{'bei': '17.00', 'le': '0.31', 'lle': '3.91', ...",CHEMBL1933288,None,CHEMBL1933288,7.80,False,http://www.openphacts.org/units/Nanomolar,1567059,=,1,True,=,None,IC50,nM,None,16.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,16.0
3,None,10840480,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.82', 'le': '0.26', 'lle': '4.45', ...",CHEMBL2012582,None,CHEMBL2012582,7.89,True,http://www.openphacts.org/units/Nanomolar,1633218,=,1,True,=,None,IC50,nM,None,13.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,13.0
4,None,10840481,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.20', 'le': '0.26', 'lle': '3.09', ...",CHEMBL509032,TAE-684,CHEMBL509032,8.11,False,http://www.openphacts.org/units/Nanomolar,1633217,=,1,True,=,None,IC50,nM,None,7.8,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,Active,20736317,[],CHEMBL4510305,LRRK2 Biochemical Assay with 5mM ATP,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@H]1CN(c2cc(-c3n[nH]c4ccc(OC5(C)CC5)cc34)nc...,None,None,CHEMBL4507291,None,2021,None,CHEMBL4098877,MLi-2,CHEMBL4098877,None,False,http://www.openphacts.org/units/Nanomolar,3359711,None,54,True,None,None,IC50,nM,None,6.3,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,6.3
199

In [ ]:
df1.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,6180232,[],CHEMBL1772597,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '16.63', 'le': '0.30', 'lle': '1.48', ...",CHEMBL1771409,None,CHEMBL1771409,4.88,False,http://www.openphacts.org/units/Nanomolar,1420091,=,1,True,=,None,IC50,nM,None,13200.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,13.2
1,None,6180234,[],CHEMBL1772599,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '20.54', 'le': '0.37', 'lle': '2.01', ...",CHEMBL1771411,None,CHEMBL1771411,5.39,False,http://www.openphacts.org/units/Nanomolar,1420093,=,1,True,=,None,IC50,nM,None,4100.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,4.1
2,None,7989381,[],CHEMBL1937075,Inhibition of human LRRK2,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,None,None,CHEMBL1933005,Bioorg. Med. Chem. Lett.,2012,"{'bei': '17.00', 'le': '0.31', 'lle': '3.91', ...",CHEMBL1933288,None,CHEMBL1933288,7.80,False,http://www.openphacts.org/units/Nanomolar,1567059,=,1,True,=,None,IC50,nM,None,16.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,16.0
3,None,10840480,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.82', 'le': '0.26', 'lle': '4.45', ...",CHEMBL2012582,None,CHEMBL2012582,7.89,True,http://www.openphacts.org/units/Nanomolar,1633218,=,1,True,=,None,IC50,nM,None,13.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,13.0
4,None,10840481,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.20', 'le': '0.26', 'lle': '3.09', ...",CHEMBL509032,TAE-684,CHEMBL509032,8.11,False,http://www.openphacts.org/units/Nanomolar,1633217,=,1,True,=,None,IC50,nM,None,7.8,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,7.8


In [ ]:
df1.to_csv('lrrk2_bioactivity_raw.csv', index=False)

##Remove missing data

In [ ]:
df2 = df1[df1.standard_value.notna()]
df2 = df2[df1.canonical_smiles.notna()]
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,6180232,[],CHEMBL1772597,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '16.63', 'le': '0.30', 'lle': '1.48', ...",CHEMBL1771409,None,CHEMBL1771409,4.88,False,http://www.openphacts.org/units/Nanomolar,1420091,=,1,True,=,None,IC50,nM,None,13200.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,13.2
1,None,6180234,[],CHEMBL1772599,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '20.54', 'le': '0.37', 'lle': '2.01', ...",CHEMBL1771411,None,CHEMBL1771411,5.39,False,http://www.openphacts.org/units/Nanomolar,1420093,=,1,True,=,None,IC50,nM,None,4100.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,4.1
2,None,7989381,[],CHEMBL1937075,Inhibition of human LRRK2,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,None,None,CHEMBL1933005,Bioorg. Med. Chem. Lett.,2012,"{'bei': '17.00', 'le': '0.31', 'lle': '3.91', ...",CHEMBL1933288,None,CHEMBL1933288,7.80,False,http://www.openphacts.org/units/Nanomolar,1567059,=,1,True,=,None,IC50,nM,None,16.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,16.0
3,None,10840480,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.82', 'le': '0.26', 'lle': '4.45', ...",CHEMBL2012582,None,CHEMBL2012582,7.89,True,http://www.openphacts.org/units/Nanomolar,1633218,=,1,True,=,None,IC50,nM,None,13.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,13.0
4,None,10840481,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.20', 'le': '0.26', 'lle': '3.09', ...",CHEMBL509032,TAE-684,CHEMBL509032,8.11,False,http://www.openphacts.org/units/Nanomolar,1633217,=,1,True,=,None,IC50,nM,None,7.8,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,Active,20736317,[],CHEMBL4510305,LRRK2 Biochemical Assay with 5mM ATP,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@H]1CN(c2cc(-c3n[nH]c4ccc(OC5(C)CC5)cc34)nc...,None,None,CHEMBL4507291,None,2021,None,CHEMBL4098877,MLi-2,CHEMBL4098877,None,False,http://www.openphacts.org/units/Nanomolar,3359711,None,54,True,None,None,IC50,nM,None,6.3,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,6.3
199

In [ ]:
df3 = df2.drop_duplicates(['canonical_smiles'])
df3

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,6180232,[],CHEMBL1772597,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '16.63', 'le': '0.30', 'lle': '1.48', ...",CHEMBL1771409,None,CHEMBL1771409,4.88,False,http://www.openphacts.org/units/Nanomolar,1420091,=,1,True,=,None,IC50,nM,None,13200.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,13.2
1,None,6180234,[],CHEMBL1772599,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,assay format,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,None,None,CHEMBL1770059,Bioorg. Med. Chem. Lett.,2011,"{'bei': '20.54', 'le': '0.37', 'lle': '2.01', ...",CHEMBL1771411,None,CHEMBL1771411,5.39,False,http://www.openphacts.org/units/Nanomolar,1420093,=,1,True,=,None,IC50,nM,None,4100.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,4.1
2,None,7989381,[],CHEMBL1937075,Inhibition of human LRRK2,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,None,None,CHEMBL1933005,Bioorg. Med. Chem. Lett.,2012,"{'bei': '17.00', 'le': '0.31', 'lle': '3.91', ...",CHEMBL1933288,None,CHEMBL1933288,7.80,False,http://www.openphacts.org/units/Nanomolar,1567059,=,1,True,=,None,IC50,nM,None,16.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,16.0
3,None,10840480,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.82', 'le': '0.26', 'lle': '4.45', ...",CHEMBL2012582,None,CHEMBL2012582,7.89,True,http://www.openphacts.org/units/Nanomolar,1633218,=,1,True,=,None,IC50,nM,None,13.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,13.0
4,None,10840481,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,None,None,CHEMBL2010643,Bioorg. Med. Chem. Lett.,2012,"{'bei': '13.20', 'le': '0.26', 'lle': '3.09', ...",CHEMBL509032,TAE-684,CHEMBL509032,8.11,False,http://www.openphacts.org/units/Nanomolar,1633217,=,1,True,=,None,IC50,nM,None,7.8,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,None,20640651,[],CHEMBL4612965,Inhibition of recombinant human GST-tagged LRR...,B,None,None,BAO_0000190,BAO_0000019,assay format,COc1cc(NS(C)(=O)=O)ccc1Nc1ncc2c(n1)N(C)c1ccccc...,None,None,CHEMBL4610122,J Med Chem,2020,"{'bei': '9.67', 'le': '0.19', 'lle': '1.16', '...",CHEMBL4641263,None,CHEMBL4641263,5.05,False,http://www.openphacts.org/units/Nanomolar,3473755,=,1,True,=,None,IC50,nM,None,8894.0,CHEMBL1075104,Homo sapiens,Leucine-rich repeat serine/threonine-protein k

##Pre-Process Data

In [ ]:
data_select = ['molecule_chembl_id','canonical_smiles','standard_value']
df4 = df3[data_select]
df4

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL1771409,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,13200.0
1,CHEMBL1771411,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,4100.0
2,CHEMBL1933288,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,16.0
3,CHEMBL2012582,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,13.0
4,CHEMBL509032,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,7.8
...,...,...,...
1990,CHEMBL4641263,COc1cc(NS(C)(=O)=O)ccc1Nc1ncc2c(n1)N(C)c1ccccc...,8894.0
1991,CHEMBL4635720,COc1cc(NS(C)(=O)=O)ccc1Nc1ncc2c(n1)N(C)c1ccccc...,683.0
1992,CHEMBL4645565,CC(O)(C#Cc1ccc(N2CCOCC2)c(Nc2ncnc3[nH]ccc23)c1...,94.6
1996,CHEMBL4560589,C[C@H]1CN(c2cc(-c3nn(C)c4ccc(OC5(C)CC5)cc34)nc...,6480.0


In [ ]:
df4.to_csv("lrrk2_bio_pre-processed.csv", index=False)

##Labeling activity

In [ ]:
df5 = pd.read_csv('lrrk2_bio_pre-processed.csv')

In [ ]:
bio_thre = []
for i in df5.standard_value:
  if float(i) <= 100:
    bio_thre.append('active')
  elif float(i) >= 300:
    bio_thre.append('inactive')
  else:
    bio_thre.append('intermediate')

In [ ]:
len(bio_thre)

1204

In [ ]:
bio_class = pd.Series(bio_thre, name='class')
df6 = pd.concat([df5, bio_class], axis=1)
df6

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL1771409,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,13200.0,inactive
1,CHEMBL1771411,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,4100.0,inactive
2,CHEMBL1933288,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,16.0,active
3,CHEMBL2012582,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,13.0,active
4,CHEMBL509032,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,7.8,active
...,...,...,...,...
1199,CHEMBL4641263,COc1cc(NS(C)(=O)=O)ccc1Nc1ncc2c(n1)N(C)c1ccccc...,8894.0,inactive
1200,CHEMBL4635720,COc1cc(NS(C)(=O)=O)ccc1Nc1ncc2c(n1)N(C)c1ccccc...,683.0,inactive
1201,CHEMBL4645565,CC(O)(C#Cc1ccc(N2CCOCC2)c(Nc2ncnc3[nH]ccc23)c1...,94.6,active
1202,CHEMBL4560589,C[C@H]1CN(c2cc(-c3nn(C)c4ccc(OC5(C)CC5)cc34)nc...,6480.0,inactive


##Save processed data

In [ ]:
df6.to_csv('lrrk2_preprocessed_data.csv', index=False)